In [4]:
import os

# 주어진 디렉토리에 있는 항목들의 이름을 담고 있는 리스트를 반환합니다.
# 리스트는 임의의 순서대로 나열됩니다.
file_path = '/aiffel/aiffel/rock_scissor_paper/set/paper/Untitled1'
file_names = os.listdir(file_path)
file_names

i = 205
for name in file_names:
    src = os.path.join(file_path, name)
    dst = str(i) + '.jpg'
    dst = os.path.join(file_path, dst)
    os.rename(src, dst)
    i += 1

In [5]:
import numpy as np
import tensorflow as tf
import glob

from tensorflow import keras
from PIL import Image


In [7]:
"""1. 이미지 전처리 과정
 이미지 전처리 (불러오기+Resize) -> 128*128로 조정
"""

def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 128x128 사이즈로 바꾸어 저장
	target_size=(128,128)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# 가위,바위,보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서 조정
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
resize_images(image_dir_path)
print("가위 이미지 resize 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper"
resize_images(image_dir_path)
print("보 이미지 resize 완료!")

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock"
resize_images(image_dir_path)
print("바위 이미지 resize 완료!")

400  images to be resized.
400  images resized.
가위 이미지 resize 완료!
400  images to be resized.
400  images resized.
보 이미지 resize 완료!
400  images to be resized.
400  images resized.
바위 이미지 resize 완료!


In [8]:

"""

2. 데이터 불러오기
load_data()이용해서 데이터 불러옴

"""

def load_data(img_path, number_of_data=1200):  # number_of_data = 전체 training set 개수./ 가위바위보 각 300개씩 총 합 1200개
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=128
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))


학습데이터(x_train)의 이미지 개수는 1200 입니다.
x_train shape: (1200, 128, 128, 3)
y_train shape: (1200,)


In [9]:

"""
3. 모델 구성하기
Mnist와 다르게 컬러이므로 채널 값 3, 128*128로 크기 설정해서
input_shape값 변경

하이퍼파라미터 값은 2^n 단위로 실험
"""

n_channel_1=128
n_channel_2=256
n_dense=64


model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(128,128,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 128)     3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 256)       295168    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 256)       0         
_________________________________________________________________
flatten (Flatten)            (None, 230400)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                14745664  
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1

In [10]:
"""

4. 모델 학습시키기

"""

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)


Epoch 1/10
38/38 [==============================] - 7s 65ms/step - loss: 151.5992 - accuracy: 0.5567
Epoch 2/10
38/38 [==============================] - 2s 49ms/step - loss: 0.1345 - accuracy: 0.9575
Epoch 3/10
38/38 [==============================] - 2s 48ms/step - loss: 0.0375 - accuracy: 0.9867
Epoch 4/10
38/38 [==============================] - 2s 49ms/step - loss: 0.0321 - accuracy: 0.9900
Epoch 5/10
38/38 [==============================] - 2s 48ms/step - loss: 0.0078 - accuracy: 0.9975
Epoch 6/10
38/38 [==============================] - 2s 48ms/step - loss: 0.0824 - accuracy: 0.9767
Epoch 7/10
38/38 [==============================] - 2s 49ms/step - loss: 0.0903 - accuracy: 0.9783
Epoch 8/10
38/38 [==============================] - 2s 49ms/step - loss: 0.0112 - accuracy: 0.9975
Epoch 9/10
38/38 [==============================] - 2s 48ms/step - loss: 3.9818e-04 - accuracy: 1.0000
Epoch 10/10
38/38 [==============================] - 2s 49ms/step - loss: 1.4359e-04 - accuracy: 1.0000

In [11]:

"""

5.test set 만들기
resize_images() 사용

"""


image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/paper"
resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/scissor"
resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/rock"
resize_images(image_dir_path)



100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.


In [13]:
#데이터 로드
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"
(x_test, y_test)=load_data(image_dir_path)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

학습데이터(x_train)의 이미지 개수는 300 입니다.
x_test shape: (1200, 128, 128, 3)
y_test shape: (1200,)


In [14]:
"""

6. test + accracy

"""

test_loss, test_accuracy = model.evaluate(x_test,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

38/38 - 1s - loss: 4.4047 - accuracy: 0.8375
test_loss: 4.404687881469727 
test_accuracy: 0.8374999761581421
